# 01 — Basics Overview (Runnable)

This notebook contains runnable examples that work without external LLM SDKs. It includes a tiny environment loader (no dependencies required), a message structure demo, prompt templating, and an improved `Memory` class that limits by characters. If you want to connect to a real LLM (Foundry or similar), see the `docs/roadmap.md` for secure configuration using a local `.env` file.

In [ ]:
# Minimal environment loader that does NOT require python-dotenv
# It reads a `.env` file at project root if present and sets os.environ entries.
import os
from pathlib import Path

def load_dotenv_if_present(dotenv_path='.env'):
    p = Path(dotenv_path)
    if not p.exists():
        print('No .env file found at', p.resolve())
        return {}
    data = {}
    with p.open() as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            if '=' not in line:
                continue
            k, v = line.split('=', 1)
            k = k.strip()
            v = v.strip().strip('"').strip('"')
            os.environ.setdefault(k, v)
            data[k] = v
    print('Loaded', len(data), 'entries from', p)
    return data

# Run loader (safe) — will not overwrite existing env vars
_loaded_env = load_dotenv_if_present()

## LLM Chat Structure (Conceptual)

LLM chat systems use a list of messages with roles: `system`, `user`, and `assistant`. The `system` message sets behavior, `user` provides instructions or queries, and `assistant` contains model outputs. We'll demonstrate how this structure maps to function calls and simple agent loops.

In [ ]:
# Example: message structure example (no external API)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Summarize the dataset columns for me."},
]
print('Messages structure:', messages)

SyntaxError: unterminated string literal (detected at line 6) (1089637377.py, line 6)

## Prompt Design (Short)
Prompts guide model behavior. Start with a clear `system` instruction, then provide context and ask concise questions. Use few-shot examples for structure when needed. The example below uses Python's `string.Template` which is built-in.

In [ ]:
# Prompt template example using built-in Template
from string import Template
template = Template('System: $system\nUser: $user_prompt')
print(template.substitute(system='You summarize tables', user_prompt='Describe columns'))

## Memory (Runnable Implementation)
Memory stores the conversation history or facts about the user. Below is a small implementation that limits stored characters (not messages) and can convert memory into a compact system prompt for future LLM calls.

In [ ]:
# Memory class that limits stored characters and can produce a system prompt
class Memory:
    def __init__(self, char_limit=500):
        self.char_limit = int(char_limit)
        self.history = []
        self._total_chars = 0
    def add(self, role, content):
        item = {"role": role, "content": content}
        self.history.append(item)
        self._total_chars += len(content)
        # Trim oldest until under limit
        while self._total_chars > self.char_limit and self.history:
            removed = self.history.pop(0)
            self._total_chars -= len(removed['content'])
    def summarize(self, max_items=5):
        return ' | '.join(m['content'] for m in self.history[-max_items:])
    def to_system_prompt(self):
        # Convert recent memory into a single system prompt string
        if not self.history:
            return ''
        return 'Memory summary: ' + self.summarize()

# Demo
mem = Memory(char_limit=200)
mem.add('user', 'I like data visualization and prefer seaborn for quick plots.')
mem.add('assistant', 'Noted — I will suggest charts and colors.')
mem.add('user', 'I often work with time series.')
print('System prompt built from memory:')
print(mem.to_system_prompt())

## Exercises
- Modify the `Memory` class to add metadata tags (e.g., `project`, `priority`) to messages and filter by tag when building a system prompt.
- Write a function that converts the memory into a structured system prompt with bullet points for the LLM.

---
End of `01_basics_overview.ipynb`. Continue with `02_code_examples.ipynb` for runnable code examples calling local dummy tools and structured explanations.